## Virtual Environment Configuration
### Install Python 3.10 on Ubuntu
Follow https://computingforgeeks.com/how-to-install-python-on-ubuntu-linux-system/ .

    sudo apt-get update
    sudo apt install software-properties-common -y
    sudo add-apt-repository ppa:deadsnakes/ppa
    sudo apt install python3.10

### Create Python 3.10 virtualenv in ~/py310

    cd ~
    pip3 install virtualenv
    virtualenv --python=/usr/bin/python3.10 py310
    source py310/bin/activate
    pip list # Show packages
    pip install --upgrade pip

### Add py310 virtualenv to Jupyter

    ipython kernel install --user --name py310

Now select the kernel when running this notebook.    

In [1]:
import sys
print (sys.version)

3.10.11 (main, Apr  5 2023, 14:15:30) [GCC 7.5.0]


# Retrieval-Augmented Generation with Pinecone & ChatGPT
More information at https://www.pinecone.io/learn/openai-gen-qa/
Github repository: https://github.com/pinecone-io/examples/tree/master/generation/generative-qa/openai/gen-qa-openai

## Install dependencies
   
    pip install -qU openai pinecone-client datasets tqdm

## Set OPENAI_API_KEY before running jupyter

You need an API key set up from: https://platform.openai.com/account/api-keys

    export OPENAI_API_KEY="secret key from site"

In [3]:
import os
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

# openai.Engine.list()  # check we have authenticated

## Test query with ChatGPT 3.5
Now run a query with ChatGPT 3.5-turbo. Using ChatCompletion.create ( https://platform.openai.com/docs/api-reference/chat/create ) you can construct a chat history (with memory) for the chatbot. (The limit is 4096 tokens.)

In [3]:
query = "What are the main components of the zero trust architecture, and what do they do?"

# now query GPT 3.5 WITHOUT context
res = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "user", "content" : query}
    ]
)

print(res['choices'][0]['message']['content'])

The main components of the zero trust architecture are:

1. Identity and access management (IAM): This component verifies the identity of users, devices, and applications before granting access to resources.

2. Multi-factor authentication (MFA): MFA adds an extra layer of security by requiring users to provide more than one form of authentication before granting access.

3. Network segmentation: This component separates the network into small, isolated segments to minimize the attack surface.

4. Micro-segmentation: Micro-segmentation limits applications and services to specific users and devices, reducing the risk of a data breach.

5. Least privileged access: This component provides users with access to only the resources they need to perform their job functions, reducing the risk of accidental or intentional data breaches.

6. Continuous monitoring and analytics: This component monitors all activity within the network, including user behavior and system access, to detect and preven

## Correcting an answer by providing more context

The answer is very good in terms of Zero Trust, but not accurate for the NIST model specifically (the NIST ZTA is defined in NIST SP 800-207 https://csrc.nist.gov/publications/detail/sp/800-207/final ).

Can we create context (provide additional information) in the chat that will allow the chatbot to answer the question correctly for the NIST ZTA? The general approach to this is **"retrieval-augmented generation"** in which we use a vector database to store documents and pull relevant text into a context that we provide the chatbot.

### Connect and Authenticate to Pinecone
Set OPENAI_API_KEY before running jupyter
You need an API key set up from: https://app.pinecone.io
     export PINECONE_API_KEY="API key from pinecone.io"
     export PINECONE_ENVIRONMENT="the environment code (next to the API key) from pinecone.io"

In [16]:
import pinecone
from tqdm.notebook import tqdm

api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

pinecone.init(api_key=api_key, enviroment=env)
# pinecone.whoami()

### Create a sample embedding so that we know the embedding length

In [5]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=["This is sample test that will determine the length"],
    engine=embed_model
)

embedding_length = len(res['data'][0]['embedding'])
embedding_length

1536

In [6]:
index_name = 'openai-nist-sp'

# Create the index if it doesn't exist already
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=embedding_length,
        metric='cosine',
        metadata_config={'indexed': ['document']}
    )

In [7]:
# connect to index
index = pinecone.Index(index_name)
# view index stats 
index_stats = index.describe_index_stats()
index_stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13}},
 'total_vector_count': 13}

In [8]:
# Read NIST SP 800-207 and add to the vector database if it is empty

entry_word_max = 1500
entries=[] # Collect our entries

if (index_stats.total_vector_count == 0):
    with open('NIST.SP.800-207.txt', 'r') as file:
        book = file.read()

    # Collect the text by paragraph into blocks of entry_word_max words
    entry = ""
    entry_word_count = 0
    for line in book.split('SECTION'):
        line_word_count = len(line.split())
        if ((line_word_count + entry_word_count) > entry_word_max):
            entries.append(entry)
            entry = ""
            entry_word_count = 0
        entry += line + " "
        entry_word_count += line_word_count

    # record last entry if not empty
    if (entry_word_count > 0):
        entries.append(entry)

len(entries)


0

In [9]:
# Now calculate the embeddings

if (len(entries) > 0):
    embeddings = openai.Embedding.create(input=entries, engine=embed_model)

In [10]:
# Now zip up the embeddings and insert!

if (len(entries) > 0):
    # Object is of the form (id, vector, meta_data)
    to_upsert = [('800-207-'+str(i), embeddings['data'][i]['embedding'],{'document':'NIST SP 800-207', 'text':entries[i]}) 
                 for i in range(len(entries)) ]
    index.upsert(vectors=to_upsert)
    

In [11]:
# Let's try a to search our index!
query = "What are the main components of the zero trust architecture, and what do they do?"
qe = openai.Embedding.create(input=[query], engine=embed_model)
res = index.query(qe['data'][0]['embedding'], top_k=2, include_metadata=True)
# res

In [12]:
context_1 = ( 
    "This is initial context for a question below\n\n" +
    "Context: \n" +
    res['matches'][0]['metadata']['text'] + "\n\n"
)
context_2 = ( 
    "This is additional context for a question below\n\n" +
    "Context: \n" +
    res['matches'][1]['metadata']['text'] + "\n\n"
)
question = (
    "Answer the question based upon the previous context. If the answer is not clear from the source, " +
    "state 'I cannot answer based upon the context.'.\n\n" +
    "Question: " + query
)

# [context_1, context_2, question]

In [13]:
# now query GPT 3.5 WITH context
res = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "user", "content" : context_1},
        {"role": "user", "content" : context_2},
        {"role": "user", "content" : question}
    ]
)

print(res['choices'][0]['message']['content'])

The main components of the zero trust architecture are policy engine (PE), policy administrator (PA), and policy enforcement point (PEP). The policy engine is responsible for the ultimate decision to grant access to a resource based on enterprise policy and external sources. The policy administrator establishes and/or shuts down the communication path between a subject and a resource, generates session-specific authentication, and is closely tied to the policy engine. The policy enforcement point enables, monitors, and terminates connections between a subject and an enterprise resource and communicates with the policy administrator to forward requests and updates policy. Additionally, there are several data sources that provide policy rules used by the policy engine, including the continuous diagnostics, industry compliance system, threat intelligence feed(s), network and system activity logs, data access policies, enterprise PKI, ID management system, and SIEM system.


In [14]:
# When we are done, delete the index
# pinecone.delete_index(index_name)